# 🏋️ Mindbody - Explorador de Clases y Asistencia

Este notebook te permite conectar con la API de Mindbody y explorar los datos de clases y asistencia de clientes.

**Requisitos:**
- API Key de Mindbody (obtener de https://developers.mindbodyonline.com/)


## 1. Configuración


In [1]:
# Importar dependencias
import pandas as pd
from datetime import datetime, timedelta
from mindbody_client import MindbodyClient, save_to_json, save_to_csv

# Configuración de display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

print("✅ Dependencias cargadas")


✅ Dependencias cargadas


In [7]:
# ══════════════════════════════════════════════════════════════════
# 🔐 CONFIGURA TUS CREDENCIALES AQUÍ
# ══════════════════════════════════════════════════════════════════

API_KEY = "48893ff74df5471281904087a6456228"  # ⬅️ Reemplaza con tu API Key

# Configuración del sandbox (para pruebas)
SITE_ID = "5738675"
STAFF_USERNAME = "mindbodysandbox99@gmail.com"
STAFF_PASSWORD = "Apitest1234"

# ══════════════════════════════════════════════════════════════════


## 2. Conectar a la API


In [8]:
# Crear cliente y autenticar
client = MindbodyClient(api_key=API_KEY, site_id=SITE_ID)

if client.authenticate(STAFF_USERNAME, STAFF_PASSWORD):
    print("🎉 Conectado exitosamente a Mindbody")
else:
    print("❌ Error de autenticación. Verifica tu API Key y credenciales.")


❌ Error de autenticación: 403 Client Error: Forbidden for url: https://api.mindbodyonline.com/public/v6/usertoken/issue
   Respuesta: {"Error":{"Message":"Before you can use this endpoint, MINDBODY must approve your developer account for live access. If you have finished testing in the sandbox and are ready to begin working with MINDBODY customers, log into your account and request to go live.","Code":"DeniedAccess"}}
❌ Error de autenticación. Verifica tu API Key y credenciales.


## 3. Descargar Clases con Asistencia


In [5]:
# Configurar rango de fechas
END_DATE = datetime.now()
START_DATE = END_DATE - timedelta(days=1)  # Últimos 9 días

print(f"📅 Buscando clases desde {START_DATE.strftime('%Y-%m-%d')} hasta {END_DATE.strftime('%Y-%m-%d')}")

# Descargar clases con asistencia
classes_data = client.get_all_classes_with_attendance(
    start_date=START_DATE,
    end_date=END_DATE
)

print(f"\n✅ Descargadas {len(classes_data)} clases")


📅 Buscando clases desde 2025-12-01 hasta 2025-12-02

📅 Buscando clases desde 2025-12-01 19:27:44.854111 hasta 2025-12-02 19:27:44.854111...
   📦 Obtenidas 15 clases...

✅ Total de clases encontradas: 15
   🔍 [1/15] Obteniendo asistencia de: Bottoms Up (2025-12-02)
   🔍 [2/15] Obteniendo asistencia de: Yoga (2025-12-02)
   🔍 [3/15] Obteniendo asistencia de: TRX Small Group (2025-12-01)
   🔍 [4/15] Obteniendo asistencia de: RPM Spinning (2025-12-01)
   🔍 [5/15] Obteniendo asistencia de: Yoga (2025-12-01)
   🔍 [6/15] Obteniendo asistencia de: crossfit level 1 (2025-12-02)
   🔍 [7/15] Obteniendo asistencia de: Beginner Swim (2025-12-02)
   🔍 [8/15] Obteniendo asistencia de: Body Pump (2025-12-01)
   🔍 [9/15] Obteniendo asistencia de: Power Yoga (2025-12-01)
   🔍 [10/15] Obteniendo asistencia de: Reformer Pilates  (2025-12-02)
   🔍 [11/15] Obteniendo asistencia de: Level 5 Bikram Yoga (2025-12-01)
   🔍 [12/15] Obteniendo asistencia de: Level 5 Bikram Yoga (2025-12-02)
   🔍 [13/15] Obteniend

## 4. Convertir a DataFrame


In [6]:
# Crear DataFrame de asistencias (una fila por cada cliente en cada clase)
attendance_records = []

for cls in classes_data:
    for attendee in cls["attendees"]:
        record = {
            "class_id": cls["class_id"],
            "class_name": cls["class_name"],
            "start_datetime": cls["start_datetime"],
            "end_datetime": cls["end_datetime"],
            "staff_name": cls["staff_name"],
            "location": cls["location"],
            "client_id": attendee["client_id"],
            "client_name": attendee["client_name"],
            "email": attendee["email"],
            "signed_in": attendee["signed_in"],
            "late_cancelled": attendee["late_cancelled"]
        }
        attendance_records.append(record)

df_attendance = pd.DataFrame(attendance_records)

print(f"📊 Total de registros de asistencia: {len(df_attendance)}")
print(f"📋 Columnas: {df_attendance.columns.tolist()}")
df_attendance.head(10)


📊 Total de registros de asistencia: 11
📋 Columnas: ['class_id', 'class_name', 'start_datetime', 'end_datetime', 'staff_name', 'location', 'client_id', 'client_name', 'email', 'signed_in', 'late_cancelled']


,class_id,class_name,start_datetime,end_datetime,staff_name,location,client_id,client_name,email,signed_in,late_cancelled
0,20384,Yoga,2025-12-02T06:20:00,2025-12-02T07:20:00,Ken Berry,Clubville,100015641,,,True,False
1,20384,Yoga,2025-12-02T06:20:00,2025-12-02T07:20:00,Ken Berry,Clubville,111,,,False,False
2,20384,Yoga,2025-12-02T06:20:00,2025-12-02T07:20:00,Ken Berry,Clubville,100015139,,,False,False
3,24651,Reformer Pilates,2025-12-02T08:00:00,2025-12-02T09:00:00,Caroline Park,Clubville,100015650,,,True,False
4,25202,Six Week Boot Camp,2025-12-01T07:30:00,2025-12-01T09:00:00,Jennifer Anderson,Clubville,100011586,,,True,False
5,25202,Six Week Boot Camp,2025-12-01T07:30:00,2025-12-01T09:00:00,Jennifer Anderson,Clubville,100011586,,,True,False
6,25202,Six Week Boot Camp,2025-12-01T07:30:00,2025-12-01T09:00:00,Jennifer Anderson,Clubville,100011586,,,True,False
7,25202,Six Week Boot Camp,2025-12-01T07:30:00,2025-12-01T09:00:00,Jennifer Anderson,Clubville,100011586,,,True,False
8,25202,Six Week Boot Camp,2025-12-01T07:30:00,2025-12-01T09:00:00,Jennifer Anderson,Clubville,100011586,,,True,False
9,25279,Squidlets 1,2025-12-02T19:00:00,2025-12-02T19:55:00,Corey Patterson,Clubville,100014010,,,False,False


## 5. Análisis de Datos


In [ ]:
# Resumen estadístico
print("=" * 60)
print("📊 RESUMEN ESTADÍSTICO")
print("=" * 60)
print(f"\n📚 Total de clases: {len(classes_data)}")
print(f"👥 Total de asistencias: {len(df_attendance)}")

if len(df_attendance) > 0:
    print(f"🧑 Clientes únicos: {df_attendance['client_id'].nunique()}")
    print(f"👨‍🏫 Instructores únicos: {df_attendance['staff_name'].nunique()}")
    print(f"📍 Ubicaciones: {df_attendance['location'].nunique()}")


In [ ]:
# Top 10 clases más populares
if len(df_attendance) > 0:
    print("\n🏆 TOP 10 CLASES MÁS POPULARES:")
    print("-" * 40)
    top_classes = df_attendance.groupby('class_name').size().sort_values(ascending=False).head(10)
    for idx, (name, count) in enumerate(top_classes.items(), 1):
        print(f"  {idx}. {name}: {count} asistencias")
else:
    print("⚠️ No hay datos de asistencia para analizar")


In [ ]:
# Clientes más frecuentes
if len(df_attendance) > 0:
    print("\n🌟 TOP 10 CLIENTES MÁS FRECUENTES:")
    print("-" * 40)
    top_clients = df_attendance.groupby(['client_id', 'client_name']).size().sort_values(ascending=False).head(10)
    for idx, ((client_id, name), count) in enumerate(top_clients.items(), 1):
        print(f"  {idx}. ID: {client_id} | {name}: {count} clases")
else:
    print("⚠️ No hay datos de asistencia para analizar")


## 6. Lista de Clientes con IDs


In [ ]:
# Lista de todos los clientes únicos con sus IDs
if len(df_attendance) > 0:
    unique_clients = df_attendance[['client_id', 'client_name', 'email']].drop_duplicates()
    unique_clients = unique_clients.sort_values('client_name')
    
    print(f"\n👥 LISTA DE CLIENTES ÚNICOS ({len(unique_clients)} total):")
    print("=" * 60)
    display(unique_clients)
else:
    print("⚠️ No hay clientes para mostrar")


## 7. Buscar Cliente Específico


In [ ]:
# Buscar un cliente específico por nombre o ID
# ⬇️ Modifica el valor de 'search_term' para buscar

search_term = ""  # ⬅️ Escribe aquí el nombre o ID del cliente

if search_term and len(df_attendance) > 0:
    # Buscar por nombre (case insensitive) o por ID
    mask = (
        df_attendance['client_name'].str.contains(search_term, case=False, na=False) |
        df_attendance['client_id'].astype(str).str.contains(search_term, na=False)
    )
    results = df_attendance[mask]
    
    print(f"🔍 Resultados para '{search_term}': {len(results)} asistencias")
    print("-" * 60)
    display(results)
else:
    print("💡 Modifica 'search_term' arriba para buscar un cliente")


## 8. Guardar Datos


In [ ]:
# Guardar datos a CSV y JSON
if len(df_attendance) > 0:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Guardar asistencias a CSV
    csv_file = f"asistencias_{timestamp}.csv"
    df_attendance.to_csv(csv_file, index=False)
    print(f"💾 Asistencias guardadas en: {csv_file}")
    
    # Guardar datos completos a JSON
    json_file = f"clases_completo_{timestamp}.json"
    save_to_json(classes_data, json_file)
    print(f"💾 Datos completos guardados en: {json_file}")
else:
    print("⚠️ No hay datos para guardar")
